# Test bed

In a console run: 

$ docker run -p 127.0.0.1:9042:9042 -i -t scylladb/scylla

or: 

$ docker run -p 127.0.0.1:9042:9042 -d cassandra

# Init

In [151]:
from cassandra.cluster import Cluster
from cassandra.util import uuid_from_time
from datetime import datetime

In [152]:
cluster = Cluster(['127.0.0.1'], port=9042, protocol_version=3)

In [153]:
session = cluster.connect()

In [154]:
cluster.metadata.cluster_name


u'Test Cluster'

In [155]:
cluster.metadata.all_hosts()

[<Host: 127.0.0.1 datacenter1>]

In [156]:
session.execute("DROP KEYSPACE glances")

In [157]:
session.execute("CREATE KEYSPACE glances WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '2' }")

In [158]:
session.set_keyspace('glances')

In [127]:
# Works with Scylla but not Cassandra...
#keyspaces = session.execute("SELECT * FROM system.schema_keyspaces")
#[k.keyspace_name for k in keyspaces]

# First data model

In [94]:
session.execute("CREATE TABLE cpu (hostname text, time timeuuid, stat map<text,float>, PRIMARY KEY (hostname, time))")

OperationTimedOut: errors={'127.0.0.1': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=127.0.0.1

In [79]:
tuuid = uuid_from_time(datetime.now())

In [39]:
session.execute(
    """
    INSERT INTO cpu (hostname, time, stat)
    VALUES (%s, %s, %s)
    """,
    ("localhost", tuuid, {"user": 30, "system": 31})
)

In [40]:
res = session.execute(
    """
    SELECT * FROM cpu
    """
)
for r in res:
    print(r)

Row(hostname=u'localhost', time=UUID('124732f4-2d88-11e6-9094-0826e0b87285'), stat=OrderedMapSerializedKey([(u'system', u'31'), (u'user', u'30')]))


In [44]:
r.stat['user']

u'30'

# Second data model

In [159]:
session.execute("CREATE TABLE localhost (plugin text, time timeuuid, stat map<text,float>, PRIMARY KEY (plugin, time))")

In [160]:
tuuid = uuid_from_time(datetime.now())

In [162]:
session.execute(
    """
    INSERT INTO localhost (plugin, time, stat)
    VALUES (%s, %s, %s)
    """,
    ("cpu", tuuid, {"user": 30, "system": 31})
)

In [164]:
res = session.execute(
    """
    SELECT * FROM localhost
    """
)
for r in res:
    print(r)

Row(plugin=u'load', time=UUID('952072ce-2e61-11e6-8015-be68a792ca4e'), stat=OrderedMapSerializedKey([(u'cpucore', 4.0), (u'history_size', 28800.0), (u'load_careful', 0.699999988079071), (u'load_critical', 5.0), (u'load_warning', 1.0), (u'min1', 0.25999999046325684), (u'min15', 0.3100000023841858), (u'min5', 0.3400000035762787)]))
Row(plugin=u'load', time=UUID('9705224c-2e61-11e6-a88d-988e2b526833'), stat=OrderedMapSerializedKey([(u'cpucore', 4.0), (u'history_size', 28800.0), (u'load_careful', 0.699999988079071), (u'load_critical', 5.0), (u'load_warning', 1.0), (u'min1', 0.23999999463558197), (u'min15', 0.3100000023841858), (u'min5', 0.3400000035762787)]))
Row(plugin=u'load', time=UUID('98e95664-2e61-11e6-9269-27a55f2a4f13'), stat=OrderedMapSerializedKey([(u'cpucore', 4.0), (u'history_size', 28800.0), (u'load_careful', 0.699999988079071), (u'load_critical', 5.0), (u'load_warning', 1.0), (u'min1', 0.23999999463558197), (u'min15', 0.3100000023841858), (u'min5', 0.3400000035762787)]))
Row(

In [101]:
session.shutdown()
cluster.shutdown()